In [1]:
import cv2
import os
import json
from tqdm import tqdm
import easyocr
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IMAGE_PATH = 'img'
VERBOSE = True

# get all the images in the folder
images = os.listdir(IMAGE_PATH)
image_paths = [os.path.join(IMAGE_PATH, image) for image in images]

In [3]:
# define the colors
# red = (255, 0, 0)
# green = (0, 176, 80)
RED = (0, 0, 255)
GREEN = (0, 176, 80)

In [4]:
# region dataclass
class Region:
    player: str
    opponent: str
    game: str
    TL: tuple
    TR: tuple
    BL: tuple
    BR: tuple

    image: np.ndarray

    DEFAULT_THRESHOLD = 2

    def __init__(self, player, opponent, game, TL, TR, BL, BR, image):
        self.player = player
        self.opponent = opponent
        self.game = game
        self.TL = TL
        self.TR = TR
        self.BL = BL
        self.BR = BR
        self.image = image
    
    def __str__(self):

        string = f'{self.game} TL: {self.TL}, TR: {self.TR}, BL: {self.BL}, BR: {self.BR}'

        return string
    
    def count_color(self, r, g, b, threshold=DEFAULT_THRESHOLD):
        mask = cv2.inRange(self.image, (b-threshold, g-threshold, r-threshold), (b+threshold, g+threshold, r+threshold))
        return cv2.countNonZero(mask)

    def count_red(self, threshold=DEFAULT_THRESHOLD):

        # convert the colors to the right format
        # r, g, b = RED
        # r = r / 255
        # g = g / 255
        # b = b / 255

        # counts how often RED is in the image
        return self.count_color(RED[0], RED[1], RED[2], threshold)

    def count_green(self, threshold=DEFAULT_THRESHOLD):
        # counts how often GREEN is in the image
        return self.count_color(GREEN[2], GREEN[1], GREEN[0], threshold)
    
    def print_colors(self, threshold=DEFAULT_THRESHOLD):
        print(f'Red: {self.count_red(threshold)}')
        print(f'Green: {self.count_green(threshold)}')

    def get_win(self):
        red = self.count_red()
        green = self.count_green()

        if red > green:
            return 0
        elif green > red:
            return 1
        else:
            # no winner
            # red and green are equal (probably both 0)
            return -1
    
    # def show_image(self):
    #     plt.imshow(self.image)
    #     plt.show()
    
    def get_image(self):
        return self.image
    
    def set_image(self, image):
        self.image = image
    
    def get_corners(self):
        return self.TL, self.TR, self.BL, self.BR

In [5]:
regions = []

with open('points.json', 'r') as f:
    regions_dict = json.load(f)

def get_image_results(image_path):

    for current_region in regions_dict:

        TL = current_region['TL']
        TR = current_region['TR']
        BL = current_region['BL']
        BR = current_region['BR']

        # get the image region
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = image.crop((TL[0], TL[1], BR[0], BR[1]))

        # convert the image to a numpy array
        image = np.array(image)

        region = Region(
            current_region['player'],
            current_region['opponent'],
            current_region['game'],
            current_region['TL'],
            current_region['TR'],
            current_region['BL'],
            current_region['BR'],
            image
        )

        regions.append(region)
    
    results = {}

    for region_index in range(0, len(regions)-1, 2):
        region1 = regions[region_index]
        region2 = regions[region_index + 1]

        region2.game = region1.game

        game = region1.game
        win = region1.get_win()

        results[game] = win
    
    return results

In [6]:
image_paths[0]

'img\\01.png'

In [7]:
full_results = {}

for image_path in tqdm(image_paths):

    results = get_image_results(image_path)
    full_results[image_path] = results

# save to results.json
with open('results.json', 'w') as f:
    json.dump(full_results, f, indent=4)

100%|██████████| 27/27 [00:00<00:00, 35.90it/s]


In [8]:
# export as csv
import csv
import re

with open('results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['round', 'player', 'opponent', 'game', 'winner', 'winner_name', 'loser_name'])

    for image_path, results in full_results.items():

        # in the image_path, find the number with regex
        # this is the round number
        round_number = re.search(r'\d+', image_path).group(0)

        # remove leading zeros
        round_number = int(round_number)

        for game, winner in results.items():
            player, opponent = game.split(':')

            winner_name = player if winner == 1 else opponent
            loser_name = player if winner == 0 else opponent

            writer.writerow([round_number, player, opponent, game, winner, winner_name, loser_name])

In [9]:
for image_path in image_paths:

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = Image.fromarray(image)

    print('------------------------------------')
    print(image_path)

    for region in regions:
        TL, TR, BL, BR = region.get_corners()
        

        region.set_image(image)

    for region_index in range(0, len(regions)-1, 2):
        player_region = regions[region_index]
        opponent_region = regions[region_index + 1]

        print(player_region.player)
        print(player_region.print_colors(), '\n')

        print(opponent_region.player)
        print(opponent_region.print_colors(), '\n')

------------------------------------
img\01.png
Kevin
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Kevin
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

Kevin
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Kevin
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Kevin
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

Kevin
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Kevin
Red: 719
Green: 1624
None 

David
Red: 719
Green: 1624
None 

Felix
Red: 719
Green: 1624
None 

Sten
Red: 719
Green: 1624
None 

Felix
R

In [10]:
# make a matplotlib plot that shows the regions and the win value and the color values
for region in regions:

    image = region.get_image()

    print(region.get_win())

    print('------------------------------------')
    print(region.player)
    print(region.opponent)
    print(region.game)
    print(region.TL)
    print(region.TR)
    print(region.BL)
    print(region.BR)
    print('------------------------------------')
    print('\n')


1
------------------------------------
Kevin
David
Kevin:David
[26, 23]
[77, 23]
[26, 73]
[77, 73]
------------------------------------


1
------------------------------------
David
Kevin
Kevin:David
[144, 21]
[190, 21]
[144, 74]
[190, 74]
------------------------------------


1
------------------------------------
Felix
Sten
Felix:Sten
[27, 110]
[82, 110]
[27, 157]
[82, 157]
------------------------------------


1
------------------------------------
Sten
Felix
Felix:Sten
[138, 106]
[189, 106]
[138, 157]
[189, 157]
------------------------------------


1
------------------------------------
Felix
David
Felix:David
[251, 23]
[303, 23]
[251, 73]
[303, 73]
------------------------------------


1
------------------------------------
David
Felix
Felix:David
[367, 21]
[412, 21]
[367, 74]
[412, 74]
------------------------------------


1
------------------------------------
Kevin
Sten
Kevin:Sten
[250, 106]
[301, 106]
[250, 153]
[301, 153]
------------------------------------


1
------

In [11]:
for region in regions:
    print(region, '\n')

    # get the red and green count
    region.print_colors(1)
    

Kevin:David TL: [26, 23], TR: [77, 23], BL: [26, 73], BR: [77, 73] 

Red: 719
Green: 1472
Kevin:David TL: [144, 21], TR: [190, 21], BL: [144, 74], BR: [190, 74] 

Red: 719
Green: 1472
Felix:Sten TL: [27, 110], TR: [82, 110], BL: [27, 157], BR: [82, 157] 

Red: 719
Green: 1472
Felix:Sten TL: [138, 106], TR: [189, 106], BL: [138, 157], BR: [189, 157] 

Red: 719
Green: 1472
Felix:David TL: [251, 23], TR: [303, 23], BL: [251, 73], BR: [303, 73] 

Red: 719
Green: 1472
Felix:David TL: [367, 21], TR: [412, 21], BL: [367, 74], BR: [412, 74] 

Red: 719
Green: 1472
Kevin:Sten TL: [250, 106], TR: [301, 106], BL: [250, 153], BR: [301, 153] 

Red: 719
Green: 1472
Kevin:Sten TL: [364, 106], TR: [415, 106], BL: [364, 155], BR: [415, 155] 

Red: 719
Green: 1472
Felix:Kevin TL: [27, 210], TR: [82, 210], BL: [27, 257], BR: [82, 257] 

Red: 719
Green: 1472
Felix:Kevin TL: [139, 210], TR: [190, 210], BL: [139, 257], BR: [190, 257] 

Red: 719
Green: 1472
David:Sten TL: [32, 292], TR: [77, 292], BL: [32, 34

In [12]:
def read_image(image_path):
    image = cv2.imread(image_path)
    return image

def analyze_image(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to get a binary image
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    print(f"Number of contours: {len(contours)}")

    # Function to extract text from a region of interest
    def extract_text(roi):
        # Convert the ROI to a PIL image
        roi_image = Image.fromarray(roi)

        # Use EasyOCR to extract text
        reader = easyocr.Reader(['en'])

        # the text is either a green W or a red L
        result = reader.readtext(roi_image, detail=0)
        if len(result) > 0:

            return result[0]
        else:

            print("No text detected")

        return "ERROR"

    # Initialize a dictionary to store the results
    results = []

    # Loop through the contours and extract the relevant information
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 50 and h > 50:  # Filter out small contours that are not relevant
            roi = binary[y:y+h, x:x+w]
            text = extract_text(roi)
            if text in ['W', 'L']:
                results.append((x, y, text))

    # Sort results by y and then by x to process each game in order
    results.sort(key=lambda k: (k[1], k[0]))

    # Organize results into pairs (each game has a pair of W and L)
    games = []
    i = 0
    while i < len(results):
        game = sorted(results[i:i+2], key=lambda k: k[2])
        games.append(game)
        i += 2

    # Display the results
    for game in games:
        print(f"Game: Player at ({game[0][0]}, {game[0][1]}) {game[0][2]} vs Player at ({game[1][0]}, {game[1][1]}) {game[1][2]}")

    # You can further process the results to get scores, matchups, etc.
    return games
